In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

# Single neuron

![experiment schematic](figs/single_cell_recording.png "Single cell recording")
![simplified system](figs/singe_cell_sim.svg)

This simplified model of a neuron is mathematically expressed with a first-order ordinary differential equation:

$$
\tau \frac{d r(t)}{dt} = -r(t) + I(t)
$$

which we can discretize in time (in steps $dt$) in order to simulate numerically using the Euler method:

$\tau \frac{r(t+dt) - r(t)}{dt} = -r(t) + I(t) \; \; \; $ so that:  $\; \; \; r(t+dt) = r(t) + \frac{dt}{\tau} \left[ -r(t) + I(t) \right]$

Here is a piece of code that simulates how a neuron with membrane time constant $\tau$ integrates a time varying input and generates a rate output $r(t)$

In [ ]:

dt = 0.001 # time step, in s
time = np.arange(0, 5, dt) # step times of the whole simulation, in s
Nt = len(time) # total number of steps in simulation
T = 0.5 # period of input current, in s

In [ ]:
input = 10*np.ones((Nt,))  # input current
# input = 1 + np.sin(2*np.pi*time/T) # input current

In [ ]:
rate = np.ones((Nt,)) # initialize firing rate at 1

tau = 0.02 # membrane time constant, in s

# simulation loop, through all time steps
for i in range(Nt-1):
    rate[i+1] = rate[i] + dt/tau*(-rate[i] + input[i])

# plot
fig, ax = plt.subplots(nrows=2)
ax[0].plot(time, input, 'k')
ax[0].set_title('Input')
ax[0].set_ylabel('current')
ax[1].plot(time, rate, 'r')
ax[1].set_title('Output')
ax[1].set_xlabel('time (s)')
ax[1].set_ylabel('rate')

Now change the code above to see how a neuron with a much longer time constant (e.g. $\tau = 1 s$) would integrate this same input. What do you see?

Now compare again the two neurons, one with short time constant ($\tau = 0.02 s$) and one with very long time constant ($\tau = 10 s$), but now have them respond to an input representing a current pulse, as per the code below:

In [ ]:
input = np.ones((Nt,))
pulsesat = int(Nt/6)
input[pulsesat] = 3

What do you see?

Now do the same thing but for an input consisting in a train of successive current pulses:

In [ ]:
input = np.ones((Nt,))
pulsesat = int(Nt/6)*np.array([1,2,3,4,5])
input[pulsesat] = 3

This is quite exciting: a single neuron can do things such as memory, or stimulus counting (integration)... but, what is a realistic membrane time constant in the brain? 20ms!

How could the brain do memory or stimulus integration if neurons have such slow time constant? Networks! Here is the simplest possible network:

# Neuron with autapse

![Autapse](figs/autapse.svg "Autapse")

Now this simplified network model of a neuron is mathematically expressed with this first-order ordinary differential equation:

$$
\tau \frac{d r(t)}{dt} = -r(t) + J r(t) + I(t)
$$

where $J$ is the strength of the self-coupling of the neuron with itself (autapse). We again discretize this equation in time (in steps $dt$) using the Euler method to get:

$r(t+dt) = r(t) + \frac{dt}{\tau} \left[ (J - 1) r(t) + I(t) \right]$

Now try simulating this equation for various values of $J$ (restricted to be non-negative, $J \ge 0$) when the neuron has a short membrane time constant (keep fixed $\tau = 0.02s$):

In [ ]:
rate = np.ones((Nt,))

tau = 0.02
J = 0.99

input = np.ones((Nt,))
pulsesat = int(Nt/6)
input[pulsesat] = 3
input = input*np.abs(1-J)

for i in range(Nt-1):
    rate[i+1] = rate[i] + dt/tau*((J - 1)*rate[i] + input[i])

fig, ax = plt.subplots(nrows=2)
ax[0].plot(time, input, 'k')
ax[0].set_title('Input')
ax[0].set_ylabel('current')
ax[1].plot(time, rate, 'r')
ax[1].set_title('Output')
ax[1].set_xlabel('time (s)')
ax[1].set_ylabel('rate');

you can play with $J$ in the above code, or make a more sophisticated plot using the **Plotly** graphics library that allows you to interact with the plot directly. Here is the code and look at the result in the cell below it:

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

dt = 0.001 # time step, in s
T = 5 # total time, in s
time = np.arange(0, T, dt) # step times of the whole simulation, in s
Nt = len(time) # total number of steps in simulation

tau = 0.02
J = 0.99

def get_traces(J):
    # initialize inputs 
    input = np.ones((Nt,))
    pulsesat = int(Nt/6)
    input[pulsesat] = 3
    input = input*np.abs(1-J)
    
    rate = np.ones((Nt,))

    for i in range(Nt-1):
        rate[i+1] = rate[i] + dt/tau*((J - 1)*rate[i] + input[i])

    return rate, input


# Create figure
#fig = go.Figure()
fig = make_subplots(rows=2, cols=1, row_heights=[0.3, 0.7])

# Add traces, one for each slider step
for step in np.arange(0.8, 1, 0.005):
    rate, input = get_traces(step)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#FF0000", width=4),
            name="",
            x = time,
            y = rate),
            row = 2, col = 1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#FF0000", width=4),
            name="",
            x = time,
            y = input,
            showlegend = False),
            row = 1, col = 1)

# Make 10th trace visible
mid = len(fig.data)//2
fig.data[mid].visible = True
fig.data[mid+1].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)//2):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}], 
        label=str(0.005 * i + 0.8) # layout attribute
    )
    step["args"][0]["visible"][2*i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][2*i+1] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=mid/2,
    currentvalue={"prefix": "Coupling J: "},
    pad={"t": 10},
    steps=steps
)]

fig.update_layout(
    sliders=sliders, height=500, width=600
)

fig.update_xaxes(title_text="", range=[0, T], row=1, col=1)
fig.update_xaxes(title_text="time (s)", range=[0, T], row=2, col=1)
fig.update_yaxes(title_text="current",  row=1, col=1)
fig.update_yaxes(title_text="rate",  row=2, col=1)


In what conditions does this network show perfect memory (i.e. it has a different rate depending on whether a stimulus was presented in the past or not)?

What happens if you set $J>1$?

So, through recurrent connections, neurons with short membrane time constants can achieve memory and integration capabilities as if they had long time constants. This is one of the fundamental principles of network function in attractor networks.

Now let's address the issues with this simple network function: fine-tuning of memory ($J=1$) and instabilities when $J > 1$. Let us consider a fundamental architectural motif in neural circuits in the cerebral cortex: neurons are either excitatory or inhibitory, and are strongly coupled with one another. 

# The E-I circuit

We thus consider two neurons, one excitatory, one inhibitory, mutually connected. The network model is now mathematically expressed with this system of coupled first-order ordinary differential equations:

$$
\tau \frac{d r_E(t)}{dt} = -r_E(t) + G_E \left[ W_{EE} r_E(t) - W_{EI} r_I(t) + I_E(t) - \theta_E \right]_{+}
$$

$$
\tau \frac{d r_I(t)}{dt} = -r_I(t) + G_I \left[ W_{IE} r_E(t) - W_{II} r_I(t) + I_I(t) - \theta_I \right]_{+} 
$$

where $W_{XY}$ are all non-negative and denote the strengths of the couplings between neuron $X$ and neuron $Y$. To ensure that firing rates are positive, we apply a linear-threshold transformation $G [\;I - \theta ]_{+}$ to the inputs, with thresholds $\theta_E$ and $\theta_I$. We again discretize this equation in time (in steps $dt$) using the Euler method to get:

$$
r_E(t+dt) = r_E(t) + \frac{dt}{\tau} \left( -r_E(t) + G_E \left[ W_{EE} r_E(t) - W_{EI} r_I(t) + I_E(t) - \theta_E \right]_+ \right)
$$

$$
r_I(t+dt) = r_I(t) + \frac{dt}{\tau} \left( -r_I(t) + G_I \left[ W_{IE} r_E(t) -W_{II} r_I(t) + I_I(t) - \theta_I \right]_+ \right)
$$

This system now is quite rich and can display a number of interesting dynamics. For instance, it can show the memory function that we discussed, but in a much more robust system. You can play with connectivity strengths in this code and convince you that this perfect memory is not subject to strict fine-tuning, thanks to the interaction of excitation and inhibition. For a detailed analysis of this system dynamics, you can explore [this article ](https://doi.org/10.7554/eLife.22425).

In [ ]:
dt = 0.001
time = np.arange(0, 5, dt)
Nt = len(time)

input = np.zeros((Nt,))
pulsesat = int(Nt/6)
input[pulsesat:pulsesat+150] = 0.4

tauE = 0.03
tauI = 0.01
WEE = 5
WEI = 1
WIE = 10
WII = 0.5
GE = 1
GI = 4

rateI = np.zeros((Nt,))
rateE = np.zeros((Nt,))

def rectify(x, threshold):
    return (x - threshold) * (x > threshold)

for i in range(Nt-1):
    rateE[i+1] = rateE[i] + dt/tauE*( -rateE[i] + GE * rectify(WEE*rateE[i] - WEI*rateI[i] + input[i], 0.35) )
    rateI[i+1] = rateI[i] + dt/tauI*( -rateI[i] + GI * rectify(WIE*rateE[i] - WII*rateI[i] , 25) )

fig, ax = plt.subplots(nrows=2)
ax[0].plot(time, input, 'k')
ax[0].set_title('Input')
ax[0].set_ylabel('current')
ax[1].plot(time, rateE, 'r', label='E-cell')
ax[1].plot(time, rateI, 'b', label='I-cell')
ax[1].set_title('Output')
ax[1].set_xlabel('time (s)')
ax[1].set_ylabel('rate')
ax[1].legend();

As an exercise, explore in this simulation the results of the phase space analysis shown in class. Try setting $W_{EE}$ outside the range of bistability, or move close and far of the bifurcation and interpret what you see.

Here is the code of the phase space analysis, if you want to generate other analyses:

In [ ]:
import brainpy as bp
import brainpy.math as bm

bm.enable_x64()
bm.set_platform('cpu')

@bp.odeint
def int_E(rE, t, rI, input=0.1, WEE=WEE, thrE=3, tauE=tauE, GE=GE):
    return - rE / tauE + GE * rectify(WEE*rE - WEI*rI + input, thrE) / tauE

@bp.odeint
def int_I(rI, t, rE, input=0.1, WIE=WIE, thrI=25, tauI=tauI, GI=GI):
    return - rI / tauI + GI * rectify(WIE*rE - WII*rI + input, thrI)  / tauI


analyzer = bp.analysis.PhasePlane2D(
    model=[int_E, int_I],
    target_vars={'rE': [0, 5], 'rI': [0, 20]},
    pars_update={'thrE': 1},
    # pars_update={'input': 0, 'WEE': 5, 'thrE': 3, 'tauE': 0.25, 'tauI': 0.01, 'GI': 4, 'thrI': 25},
    resolutions=0.05,
)
analyzer.plot_vector_field()
analyzer.plot_nullcline(coords=dict(rI='rE-rI'),
                        x_style={'fmt': '-'},
                        y_style={'fmt': '-'})
analyzer.plot_fixed_point()

plt.gca().set_box_aspect(1)
plt.ylabel('$rate_I$')
plt.xlabel('$rate_E$')
plt.tight_layout()


In [ ]:

analyzer = bp.analysis.Bifurcation2D(
  model=[int_E, int_I],
  target_vars={'rE': [0., 10], 'rI': [0., 20.]},
#  target_pars={'thrE': [2., 15.]},
#  resolutions={'thrE': 0.3},
  target_pars={'WEE': [2., 6.]},
  resolutions={'WEE': 0.05},
)

analyzer.plot_bifurcation(num_rank=50)

plt.close();
plt.gca().set_box_aspect(1)
plt.xlabel('WEE')
plt.ylabel('$rate_E$')
plt.tight_layout()

Again, here we have the interactive plot to explore the effect of $G_E$ on the stability of the network activity:

In [ ]:
dt = 0.001 # time step, in s
T = 1 # total time, in s
time = np.arange(0, T, dt) # step times of the whole simulation, in s
Nt = len(time) # total number of steps in simulation

input = np.zeros((Nt,))
pulsesat = int(Nt/3)
input[pulsesat:pulsesat+50] = 0.4

tauE = 0.03
tauI = 0.01
WEE = 5
WEI = 1
WIE = 10
WII = 0.5
GE = 1
GI = 4


def rectify(x, threshold):
    return (x - threshold) * (x > threshold)

def get_traces(GE):
    
    rateI = np.zeros((Nt,))
    rateE = np.zeros((Nt,))

    for i in range(Nt-1):
        rateE[i+1] = rateE[i] + dt/tauE*( -rateE[i] + GE * rectify(WEE*rateE[i] - WEI*rateI[i] + input[i], 0.35) )
        rateI[i+1] = rateI[i] + dt/tauI*( -rateI[i] + GI * rectify(WIE*rateE[i] - WII*rateI[i] , 25) )

    return rateE, rateI, input


# Create figure
#fig = go.Figure()
fig = make_subplots(rows=2, cols=1, row_heights=[0.3, 0.7])

# Add traces, one for each slider step
for step in np.arange(0.5, 1.5, 0.005):
    rateE, rateI, input = get_traces(step)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#FF0000", width=4),
            name="E-pop",
            x = time,
            y = rateE),
            row = 2, col = 1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#0000FF", width=4),
            name="I-pop",
            x = time,
            y = rateI),
            row = 2, col = 1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#000000", width=4),
            name="",
            x = time,
            y = input,
            showlegend = False),
            row = 1, col = 1)

# Make 10th trace visible
mid = len(fig.data)//3
fig.data[mid].visible = True
fig.data[mid+1].visible = True
fig.data[mid+2].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)//3):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}], 
        label=str(np.around(0.005 * i + 0.5, 2)) # layout attribute
    )
    step["args"][0]["visible"][3*i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][3*i+1] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][3*i+2] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=mid/2,
    currentvalue={"prefix": "Coupling GE: "},
    pad={"t": 10},
    steps=steps
)]

fig.update_layout(
    sliders=sliders, height=500, width=600
)

fig.update_xaxes(title_text="", range=[0, T], row=1, col=1)
fig.update_xaxes(title_text="time (s)", range=[0, T], row=2, col=1)
fig.update_yaxes(title_text="current",  row=1, col=1)
fig.update_yaxes(title_text="rate",  row=2, col=1)


## Advanced topic 1: Inhibition-stabilized network

How does this network respond to an excitatory input pulse applied to the excitatory population? and to the inhibitory population? 

In [ ]:
dt = 0.0005
time = np.arange(0, 5, dt)
Nt = len(time)

# initialize all inputs to zero
inputE = np.zeros((Nt,))
inputI = np.zeros((Nt,))

# first pulse to go into the non-zero solution
pulsesat = int(Nt/6)
inputE[pulsesat:pulsesat+150] = 0.4 

# now let's apply a pulse in the middle of the simulation to either the excitatory or the inhibitory neuron
# PLAY WITH THE TWO POSSIBLE INPUTS inputI AND inputE
pulsesat = int(3*Nt/6)
inputI[pulsesat:pulsesat+300] = 4
#inputE[pulsesat:pulsesat+300] = 4

tauE = 0.01
tauI = 0.002
WEE = 5
WEI = 1
WIE = 10
WII = 0.5
GE = 1
GI = 4

rateI = np.zeros((Nt,))
rateE = np.zeros((Nt,))

def rectify(x, threshold):
    return (x - threshold) * (x > threshold)

for i in range(Nt-1):
    rateE[i+1] = rateE[i] + dt/tauE*( -rateE[i] + GE * rectify(WEE*rateE[i] - WEI*rateI[i] + inputE[i], 0.35) )
    rateI[i+1] = rateI[i] + dt/tauI*( -rateI[i] + GI * rectify(WIE*rateE[i] - WII*rateI[i] + inputI[i], 25) )

fig, ax = plt.subplots(nrows=2)
ax[0].plot(time, inputE, 'r')
ax[0].plot(time, inputI, 'b')
ax[0].set_title('InputE')
ax[0].set_ylabel('current')
ax[1].plot(time, rateE, 'r', label='E-cell')
ax[1].plot(time, rateI, 'b', label='I-cell')
ax[1].set_title('Output')
ax[1].set_xlabel('time (s)')
ax[1].set_ylabel('rate')
ax[1].legend()

What do you see? Do you see a paradox? This network regime has been named "inhibition-stabilized network" or ISN. You can learn more about it in [this article](https://doi.org/10.1016/j.neuron.2009.03.028). This regime of operation is now considered to apply quite generally to neural circuits in the cerebral cortex (see [this article](https://doi.org/10.7554/eLife.54875)).

If you were wondering, there are other EI networks very similar to this one that do not behave in this paradoxical way. Take a look for instance to the classic Wilson-Cowan implementation of the EI network:

$$

\small \tau_E \frac{d r_E(t)}{dt} = \small -r_E(t) + [1-r_E(t)] F_E\left[ W_{EE} r_E(t) - W_{EI} r_I(t) \right] \\

\small \tau_I \frac{d r_I(t)}{dt} = \small -r_I(t) + [1-r_I(t)] F_I\left[ W_{IE} r_E(t) - W_{II} r_I(t) \right] \\
$$

In [ ]:
import brainpy as bp
import brainpy.math as bm

dt = 0.001 # time step, in s
T = 3 # total time, in s
time = np.arange(0, T, dt) # step times of the whole simulation, in s
Nt = len(time) # total number of steps in simulation


WEE = 12
WEI = 4
WIE = 13
WII = 11

tauE = 0.01
tauI = 0.01

GE = 1.2
GI = 1

thrE = 2.8
thrI = 4

def F (x, a, theta):
    return 1 / (1 + bm.exp(-a * (x - theta))) - 1 / (1 + bm.exp(a * theta))

def get_traces(inp_to=0):
    inputE = np.zeros((Nt,))
    pulsesat = int(Nt/3)
    inputE[pulsesat:pulsesat+50] = 4
    inputI = np.zeros((Nt,))
    inputI[pulsesat:pulsesat+50] = 4

    pulsesat = int(2*Nt/3)
    if inp_to==1:
        inputE[pulsesat:pulsesat+200] = 4
    elif inp_to==-1:
        inputI[pulsesat:pulsesat+200] = 2
    
    rateI = np.zeros((Nt,))
    rateE = np.zeros((Nt,))

    for i in range(Nt-1):
        rateE[i+1] = rateE[i] + dt/tauE*( -rateE[i] + (1 - rateE[i]) * F(WEE*rateE[i] - WEI*rateI[i] + inputE[i], GE, thrE) )
        rateI[i+1] = rateI[i] + dt/tauI*( -rateI[i] + (1 - rateI[i]) * F(WIE*rateE[i] - WII*rateI[i] + inputI[i], GI, thrI) )

    return rateE, rateI, inputE, inputI


# Create figure
#fig = go.Figure()
fig = make_subplots(rows=2, cols=1, row_heights=[0.3, 0.7], shared_xaxes=True)

# Add traces, one for each slider step
for step in list([-1, 0, 1]):
    rateE, rateI, inputE, inputI = get_traces(step)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#FF0000", width=4),
            name="E-pop",
            x = time,
            y = rateE,
            showlegend = False),
            row = 2, col = 1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#0000FF", width=4),
            name="I-pop",
            x = time,
            y = rateI,
            showlegend = False),
            row = 2, col = 1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#550000", width=4),
            name="",
            x = time,
            y = inputE,
            showlegend = False),
            row = 1, col = 1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#000055", width=4),
            name="",
            x = time,
            y = inputI,
            showlegend = False),
            row = 1, col = 1)

# Make 10th trace visible
mid = len(fig.data)//3
fig.data[mid].visible = True
fig.data[mid+1].visible = True
fig.data[mid+2].visible = True
fig.data[mid+3].visible = True


fig.update_layout(
    height=400, width=500,
    margin=dict(l=20, r=20, t=20, b=20),
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.57,
            y=1.2,
            buttons=list([
                dict(label="None",
                     method="update",
                     args=[{"visible": [False, False, False, False, True, True, True, True, False, False, False, False]},
                           {"title": "No stim"}]),
                dict(label="StimE",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, False, True, True, True, True]},
                           {"title": "Stim E"}]),
                dict(label="StimI",
                     method="update",
                     args=[{"visible": [True, True, True, True, False, False, False, False, False, False, False, False]},
                           {"title": "Stim I"}]),
            ]),
        )
    ])

fig.update_xaxes(title_text="", range=[0, T], row=1, col=1)
fig.update_xaxes(title_text="time (s)", range=[0, T], row=2, col=1)
fig.update_yaxes(title_text="current",  row=1, col=1)
fig.update_yaxes(title_text="rate",  row=2, col=1)

config = {'displayModeBar': False}

fig.show(config=config)

# Double-well attractor

![](figs/doublewell.svg)

Now this starts getting complicated. Following the previous schematic for the EI network above, you could certainly write the code for this 3-neuron network (in practice, we think of each unit in the network as a population of neurons, so we talk of a 3-population network). This is an exercise for you to try on your own.

However, we usually simplify things to deal with only two dynamical variables. Here, we can take advantage of the fact that inhibitory neurons have a faster time constant to assume that their equation relaxes more quickly to the steady state, so we freeze the firing rate of the inhibitory population to the steady state of its dynamics given the momentary rates of the two excitatory populations. If you are interested in these derivations, you can check them in [this book chapter](https://neuronaldynamics.epfl.ch/online/Ch16.S3.html). The resulting equations are:

$$
\tau_E \frac{d I_{E,1}}{dt} = - I_{E1} + (W_{EE} - \alpha) g_E(I_{E1}) - \alpha g_E(I_{E2}) + S_{1}
$$

$$
\tau_E \frac{d I_{E,2}}{dt} = - I_{E2} + (W_{EE} - \alpha) g_E(I_{E2}) - \alpha g_E(I_{E1}) + S_{2} 
$$

where $\alpha = -\gamma W_{EI}W_{IE}$ represents the effective inhibitory coupling between excitatory populations (via the inhibitory population), and $S_1$/$S_2$ are the inputs arriving to each excitatory population. We can simplify the parametrization of these equations by defining $J_E=W_{EE}-\alpha$ and $J_I=\alpha$ to get the [Wong and Wang model](https://doi.org/10.1523/JNEUROSCI.3733-05.2006):


$$
\tau_E \frac{d I_{E,1}}{dt} = - I_{E1} + J_E g_E(I_{E1}) - J_I g_E(I_{E2}) + S_{1}
$$

$$
\tau_E \frac{d I_{E,2}}{dt} = - I_{E2} + J_E g_E(I_{E2}) - J_I g_E(I_{E1}) + S_{2} 
$$

Note that this model formulation is slightly different to what we have been doing so far in this workshop: now our dynamical variables are the inputs and not the rates of the populations. To plot the rates, we have to use $r_{E} = g_E(I_E)$.

These equations can be discretized to obtain this simulation code:

In [ ]:
dt = 0.0002
time = np.arange(0, 10, dt)
Nt = len(time)

# initialize inputs 
drive0 = 0.05
inputE1 = drive0*np.ones((Nt,))
inputE2 = drive0*np.ones((Nt,))


# current pulse into one of the two populations
pulsesat = int(Nt/6)
inputE1[pulsesat:pulsesat+150] = 0.4 

tauE = 0.01
JE = 1.1
JI = 1.8

inpE1 = -0.005*np.ones((Nt,))
inpE2 = -0.005*np.ones((Nt,))

def curr_to_rate(x):
    return (1+np.tanh(x-0.5))/2

for i in range(Nt-1):
    inpE1[i+1] = inpE1[i] + dt/tauE*(-inpE1[i] + JE*curr_to_rate(inpE1[i]) - JI*curr_to_rate(inpE2[i]) + inputE1[i] )
    inpE2[i+1] = inpE2[i] + dt/tauE*(-inpE2[i] + JE*curr_to_rate(inpE2[i]) - JI*curr_to_rate(inpE1[i]) + inputE2[i] )

rateE1 = curr_to_rate(inpE1)
rateE2 = curr_to_rate(inpE2)

fig, ax = plt.subplots(nrows=2)
ax[0].plot(time, inputE1, 'r')
ax[0].plot(time, inputE2, 'm')
ax[0].set_title('InputE')
ax[0].set_ylabel('current')
ax[1].plot(time, rateE1, 'r', label='Population 1')
ax[1].plot(time, rateE2, 'm', label='Population 2')
ax[1].set_title('Output')
ax[1].set_xlabel('time (s)')
ax[1].set_ylabel('rate')
ax[1].legend()

What happens if you send the input current pulse to population 2 instead of population 1? Argue about how this model can be used to remember specific events that happened in the recent past. 

These discrete attractor models have been applied to model working memory for objects. In this case there are just two possible memories, because there are two populations, but this can be generalized to an arbitrary number of discrete memories. If you want to learn more about these models, you can check [this paper](https://www.nature.com/articles/s41586-019-0919-7) on experimental evidence supporting it.

Also, notice how this network now responds to input by establishing a competition between the two populations: if one wins, the other one loses. This is what we call "winner-take-all" dynamics and it is thought also to be a mechanism underlying multiple brain computations from sensory perception to decision making.

Here is the code for the phase space analysis of this discrete memory network. Explore how the first graph changes with the parameter `drive` (a uniform input to all the network) and use this to make sense of the second graph:

In [ ]:
drive = drive0

@bp.odeint
def int_r1(r1, t, r2, drive=drive):
    fct = 2*r1*(1.-r1)/tauE
    cnv = 0.5 + bm.atanh(2*r1 - 1.)
    return (- cnv + JE*r1 - JI*r2 + drive) *fct

@bp.odeint
def int_r2(r2, t, r1, drive=drive):
    fct = 2*r2*(1.-r2)/tauE
    cnv = 0.5 + bm.atanh(2*r2 - 1.)
    return (- cnv + JE*r2 - JI*r1 + drive) *fct


analyzer = bp.analysis.PhasePlane2D(
    model=[int_r1, int_r2],
    target_vars={'r1': [0, 1], 'r2': [0, 1]},
    # pars_update={'drive': -0.5},
    resolutions=0.0005,
)
analyzer.plot_vector_field()
analyzer.plot_nullcline(coords=dict(r2='r2-r1'),
                        x_style={'fmt': '-'},
                        y_style={'fmt': '-'})
analyzer.plot_fixed_point()
plt.gca().set_box_aspect(1)
plt.tight_layout()


In [ ]:
@bp.odeint
def int_s1(s1, t, s2, drive=drive):
    crE1 = (1 + bm.tanh(s1 - 0.5))/2.
    crE2 = (1 + bm.tanh(s2 - 0.5))/2.
    return - s1 / tauE + JE*crE1 / tauE - JI*crE2 / tauE + drive / tauE

@bp.odeint
def int_s2(s2, t, s1, drive=drive):
    crE1 = (1 + bm.tanh(s1 - 0.5))/2.
    crE2 = (1 + bm.tanh(s2 - 0.5))/2.
    return - s2 / tauE + JE*crE2 / tauE - JI*crE1 / tauE + drive / tauE

analyzer = bp.analysis.Bifurcation2D(
  model=[int_s1, int_s2],
  target_vars={'s1': [-1.5, 1.5], 's2': [-1.5, 1.5]},
  target_pars={'drive': [-0.2, 0.5]},
  resolutions={'drive': 0.01},
)

analyzer.plot_bifurcation(num_rank=50)

plt.close();
plt.gca().set_box_aspect(1)
plt.xlabel('drive')
plt.ylabel('current E1 (s1)')
plt.tight_layout()

One very useful way to think about this dynamics is to visualize it as the evolution of a ball that bounces down in a hilly landscape. This is more than just a visual analogy for networks that satisfy perfect symmetry (i.e. the connection from neuron X to neuron Y is equal to the connection from neuron Y to neuron X). For these kinds of networks this analogy is mathematically exact (see for example an explanation [here](https://neuronaldynamics.epfl.ch/online/Ch16.S4.html)) and this hilly landscape is what we call "energy". 

The energy landscape for this network can be derived analytically as described by [Gerstner et al, 2014](https://neuronaldynamics.epfl.ch/online/Ch16.S4.html):

$$
E(r_1, r_2) = -\frac{J_E}{2} (r_1^2 + r_2^2) + J_I r_1 r_2 - S (r_1 + r_2) + \int_0^{r_1} g_E^{-1}(x) dx + \int_0^{r_2} g_E^{-1}(x) dx
$$

Here are two different visualizations:

In [ ]:
colorscale = "Viridis"

np.seterr(divide = 'ignore') 

def integral(rate):
    lim = 2*rate - 1
    return 0.5*rate + 0.5* (lim*np.arctanh(lim) + 0.5*np.log(np.abs(1-lim*lim)) -0.7);

def get_energy(drive1, drive2):
    energy = np.zeros((100,100))    
    rate1 = np.linspace(0,1,100)
    rate2 = np.linspace(0,1,100)
    for i, r1 in enumerate(rate1):
        for j, r2 in enumerate(rate2):
            energy[i,j] = -0.5*JE*(r1**2 + r2**2) + JI*r1*r2 - (drive1*r1 + drive2*r2) + integral(r1) + integral(r2)

    return energy, rate1, rate2

energy, rate1, rate2 = get_energy(drive, drive)

# create figure
layout = go.Layout(hovermode=False)
fig = go.Figure(layout=layout)

# Add surface trace
potential=energy
potential[potential>0.25]=np.nan
fig.add_trace(go.Surface(z=potential, 
    x=rate1, 
    y=rate2,
    contours = {
        "z": {"show": True, "start": -0.15, "end": 0.25, "size": 0.01, "color":"white"}
    },
    colorscale=colorscale,
    colorbar_title_text='energy'))

fig.update_traces(contours_z=dict(show=True, usecolormap=True,highlightcolor="limegreen", project_z=False))

# Update plot sizing
fig.update_layout(
    width=500, height=450,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)


fig.update_scenes(xaxis_title_text='rate population 1',  
                  yaxis_title_text='rate population 2',  
                  zaxis_title_text='energy')


In [ ]:
def integral(rate):
    lim = 2*rate - 1
    return 0.5*rate + 0.5* (lim*np.arctanh(lim) + 0.5*np.log(np.abs(1-lim*lim)) -0.7);

def get_energy(drive1, drive2):
    energy = np.zeros((100,100))    
    rate1 = np.linspace(0,1,100)
    rate2 = np.linspace(0,1,100)
    for i, r1 in enumerate(rate1):
        for j, r2 in enumerate(rate2):
            energy[i,j] = -0.5*JE*(r1**2 + r2**2) + JI*r1*r2 - (drive1*r1 + drive2*r2) + integral(r1) + integral(r2)

    return energy, rate1, rate2

layout = go.Layout(hovermode=False)
fig = go.Figure(layout=layout)
      
energy, rateE1, rateE2 = get_energy(drive, drive)
# energy[energy>-0.15]=np.nan
fig.add_trace(
        go.Contour(
            visible=True,
            x = rateE1,
            y = rateE2,
            z = energy,
            contours = {"start": -0.3025, "end": -0.276, "size": 0.002, "coloring":"lines"},
            colorscale=colorscale,
            colorbar_title_text='energy'
            )
        )

fig.update_layout(height=450, width=500,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)


fig.update_xaxes(title_text="rate 1", range=[0, 1])
fig.update_yaxes(title_text="rate 2", range=[0, 1])
fig.update_scenes(xaxis_title_text='rate 1',  
                  yaxis_title_text='rate 2')

config = {'displayModeBar': False}

fig.show(config=config)


## The double-well model in decision making

Now let's explore this network in a different situation: we do not establish a difference between the external inputs to the two populations but we add random independent noise to the currents of the two populations at each time step in the simulated dynamics. This represents internal noise of the brain. Explore what happens in this simulation by running it repeatedly over several "trials". Notice also that we change the overall drive to the network during the simulation, but keeping it equal for the two populations. What does this change in external drive achieve?

In [ ]:
dt = 0.0005
time = np.arange(0, 10, dt)
Nt = len(time)

# initialize inputs 
drive0 = 0.
inputE1 = drive0*np.ones((Nt,))
inputE2 = drive0*np.ones((Nt,))

drive = 0.1
inputE1[Nt//4:] = drive
inputE2[Nt//4:] = drive

tauE = 0.05
JE = 1.1
JI = 1.8

inpE1 = -0.005*np.ones((Nt,))
inpE2 = -0.005*np.ones((Nt,))

def curr_to_rate(x):
    return (1+np.tanh(x-0.5))/2

sigma = 0.3

for i in range(Nt-1):
    noise = sigma * np.random.randn()
    inpE1[i+1] = inpE1[i] + dt/tauE*(-inpE1[i] + JE*curr_to_rate(inpE1[i]) - JI*curr_to_rate(inpE2[i]) + inputE1[i] + noise)
    noise = sigma * np.random.randn()
    inpE2[i+1] = inpE2[i] + dt/tauE*(-inpE2[i] + JE*curr_to_rate(inpE2[i]) - JI*curr_to_rate(inpE1[i]) + inputE2[i] + noise)

rateE1 = curr_to_rate(inpE1)
rateE2 = curr_to_rate(inpE2)

fig, ax = plt.subplots(nrows=2)
ax[0].plot(time, inputE1, 'r')
ax[0].plot(time, inputE2, 'm')
ax[0].set_title('InputE')
ax[0].set_ylabel('current')
ax[1].plot(time, rateE1, 'r', label='Population 1')
ax[1].plot(time, rateE2, 'm', label='Population 2')
ax[1].set_title('Output')
ax[1].set_xlabel('time (s)')
ax[1].set_ylabel('rate')
ax[1].legend()

Which of the two populations wins now the competition? How is that determined?

The advantage of having just two dynamical variables is that we can visualize the dynamics in the phase space:

In [ ]:
dt = 0.0005
time = np.arange(0, 10, dt)
Nt = len(time)

# initialize inputs 
drive0 = 0.
inputE1 = drive0*np.ones((Nt,))
inputE2 = drive0*np.ones((Nt,))

drive = 0.1
inputE1[Nt//4:] = drive
inputE2[Nt//4:] = drive

tauE = 0.05
JE = 1.1
JI = 1.8

inpE1 = -0.005*np.ones((Nt,))
inpE2 = -0.005*np.ones((Nt,))

def curr_to_rate(x):
    return (1+np.tanh(x-0.5))/2

sigma = 0.3

for i in range(Nt-1):
    noise = sigma * np.random.randn()
    inpE1[i+1] = inpE1[i] + dt/tauE*(-inpE1[i] + JE*curr_to_rate(inpE1[i]) - JI*curr_to_rate(inpE2[i]) + inputE1[i] + noise)
    noise = sigma * np.random.randn()
    inpE2[i+1] = inpE2[i] + dt/tauE*(-inpE2[i] + JE*curr_to_rate(inpE2[i]) - JI*curr_to_rate(inpE1[i]) + inputE2[i] + noise)

rateE1 = curr_to_rate(inpE1)
rateE2 = curr_to_rate(inpE2)

plt.scatter(rateE1, rateE2, c=time, s=4, cmap='cool')
plt.colorbar(label='time')
plt.xlim([0,1])
plt.ylim([0,1])
plt.axline((1, 1), slope=1, color='k', ls='--')
plt.xlabel('rate population 2')
plt.ylabel('rate population 1');

try running several times the previous cell and see how the network dynamics changes from trial to trial. Can you make sense of this dynamics? Where does the network start and where does it evolve to towards the end of the trial?

In [ ]:
np.seterr(divide = 'ignore') 

energy = np.zeros((100,100))

def integral(rate):
    lim = 2*rate - 1
    return 0.5*rate + 0.5* (lim*np.arctanh(lim) + 0.5*np.log(np.abs(1-lim*lim)) -0.7);

for i in range(100):
    r1 = i/100
    for j in range(100):
        r2 = j/100
        energy[i,j] = -0.5*JE*(r1**2 + r2**2) + JI*r1*r2 - (drive*r1 + drive*r2) + integral(r1) + integral(r2)

plt.contour(energy, extent=[0, 1, 0, 1], levels=500)
plt.colorbar(label='energy')
plt.xlabel('rate population 2')
plt.ylabel('rate population 1')


plt.scatter(rateE1, rateE2, c=time, s=4, cmap="cool")
plt.colorbar(label='time')
plt.xlim([0,1])
plt.ylim([0,1])
plt.axline((1, 1), slope=1, color='k', ls='--')
plt.xlabel('rate population 2')
plt.ylabel('rate population 1');

As you can see, the trajectories in phase space that we saw before are falling down this landscape towards the two "wells" of minimal energy. Based on this energy picture, we often call this dynamics "double-well attractor", and it is a possible circuit mechanism both for working memory and decision making.

The energy landscape can also help us understand what happens in the model when we imbalance the inputs to the two populations. For instance, if the drive to population 2 is stronger than the drive to population 1, simulating the case in which stronger evidence is presented for stimulus 2 than for stimulus 1. This biases the competition between the two populations in favor of population 2 by making the well of population 1 much shallower, or even making it disappear:

In [ ]:
energy = np.zeros((100,100))
drive1 = 0.08
drive2 = 0.12

def integral(rate):
    lim = 2*rate - 1
    return 0.5*rate + 0.5* (lim*np.arctanh(lim) + 0.5*np.log(np.abs(1-lim*lim)) -0.7)

for i in range(100):
    r1 = i/100
    for j in range(100):
        r2 = j/100
        energy[i,j] = -0.5*JE*(r1**2 + r2**2) + JI*r1*r2 - (drive1*r1 + drive2*r2) + integral(r1) + integral(r2)

plt.contour(energy, extent=[0, 1, 0, 1], levels=500)
plt.colorbar(label='energy')
plt.xlabel('rate population 2')
plt.ylabel('rate population 1');

If you want to learn more about how this model has been applied to decision making, you can read [this paper](https://www.jneurosci.org/content/26/4/1314.short) by Wong and Wang.

# Ring attractor

We have now seen a model that can do interesting computations on a few numbers of discrete items, whether memories or decisions. What about storing a continuous quantity, like a length or an angle? For this purpose, the main model studied in computational neuroscience is the ring attractor model.

![ring model](figs/ring.svg)

In this model, neurons are disposed on a ring and they are connected following a very precise connectivity scheme: neurons being close on the ring excite them more than neurons being more distant on the ring, which eventually inhibit each other effectively. Importantly, this pattern of connectivity is assumed to be translationally invariant, i.e. the strength of the connection between neuron N and neuron N + M  in the ring is exactly the same as the connection between neuron L and neuron L + M, for any N, M and L. Below I show you how to build such a connectivity scheme easily in Python. Looking at the graph below, convince yourself that this connectivity has the properties mentioned above.

In [ ]:
Nn = 128
x= np.linspace(-np.pi, np.pi, Nn, endpoint=False)

cs = np.cos(x)
sn = np.sin(x)

J0 = -3.2
J2 = 8.5
J = J0 + J2* (np.outer(cs,cs) + np.outer(sn, sn))

plt.imshow(J, origin="lower")
plt.xlabel('to neuron')
plt.ylabel('from neuron')
plt.colorbar(label="connection strength");

Now let's run the dynamics of a network with this connectivity. Instead of evolving two variables, one for each of two neural populations, as in the double-well model before, we now will evolve a whole vector of rates `rate` containging as many rate variables as neurons in the ring.

In [ ]:
dt=0.01
time = np.arange(0,5,dt)
Nt = len(time)

rate = np.zeros((Nt,Nn)) # vector of rates through time

In [ ]:
# input-output function for all cells, as used previously (Brunel, Cereb Cortex 13:1151, 2003)
def fI(x):
    return x*x*(x>0)*(x<1) + np.sqrt(np.abs(4*x-3))*(x>=1)

In [ ]:
stim = 90 * np.pi/180.

I = 4*(1 - 0.1 + 0.1 * np.cos(x - stim)) # stimulus current

tau = 0.1

for i in range(Nt-1):
    if (i>100)&(i<150): 
        input=I #transient input at the beginning of the simulation
    else:
        input=0
    network_inputs = np.dot(J, rate[i])/Nn # this is the network step, where currents to each neuron are computed from the connectivity J and the rates
    rate[i+1] = rate[i] + dt/tau * (-rate[i] + fI(network_inputs + input))

Let's plot the result of the simulation (vertical dotted lines mark the period of stimulus presentation):

In [ ]:
plt.figure(figsize=(8,3))
plt.imshow(rate.T, extent=[0,5,-180,180], aspect="auto", origin="lower")
plt.axvline(100*dt, color='w', ls=':')
plt.axvline(150*dt, color='w', ls=':')
plt.ylabel('neuron (deg)')
plt.xlabel('time (s)')
plt.colorbar(label='firing rate');

Play with the previous code changing: 
1) the strength of the connectivity $J_2$ (try reducing it to $J_2 = 7$)
2) the location where the stimulus is presented (`stim`)

What do you observe? Is this always a good system to store continuous memories?

If we now take a snapshot of the network activity at the end of the simulation, you can see why we also often call this network a *bump attractor network*.

In [ ]:
plt.plot(np.rad2deg(x), rate[-1]);
plt.xlabel('neuron (deg)')
plt.ylabel('rate');

Also in this network it is interesting to explore how it reacts when it is subject to noisy inputs, simulating the embedding of the circuit in a larger system with many uncontroled and unpredictable components.

In [ ]:
nsims=100
dt=0.01
time = np.arange(0,5,dt)
Nt = len(time)

# routine to extract population vectors from matrix of rates
vecs = np.cos(x) + 1j*np.sin(x)
vecs = np.outer(np.ones((1,Nt)),vecs)
def decode(rate):
    res = np.sum(vecs*rate, axis=1)
    return np.angle(res)

endpoints = np.zeros((nsims,))

plt.figure(figsize=(8,4))

# run the simulation multiple times
for n in range(nsims):

    I = 2*np.exp(4*np.cos(x))

    rate = np.zeros((Nt,Nn))

    tau = 0.1

    for i in range(Nt-1):
        if (i>100)&(i<150):  
            input=I
        else:
            input=0
        network_inputs = np.dot(J, rate[i])/Nn
        noise = 0.3*np.random.randn(Nn,1).flatten()
        rate[i+1] = rate[i] + dt/tau * (-rate[i] + fI(network_inputs + input + noise))
    
    trace = decode(rate)

    plt.plot(time, np.rad2deg(trace))

    endpoints[n] = trace[-1]


plt.ylim([-20, 20])
plt.xlabel('time (s)')
plt.ylabel('neuron (deg)');


How do you interpret the results of this simulation? What happens with memories in this network as we make the retention interval or delay period longer and longer? This is a prediction of the model that was effectively validated in neural data of the monkey prefrontal cortex. You can check it out [here](http://neurophysics.ucsd.edu/courses/physics_171/Wimmer_Compte_reprint.pdf).